# argparse 库

[`argparse`库]()是Python标准库，用来解析命令行解析，是编写友好命令行程序的重要工具之一。使用`argparse`模块前，需要先行导入：

In [6]:
import argparse

## 计算机中的程序

大家常常期望一个程序功能强大，使用要灵活方便，尽管很多时候是有些自相矛盾的。

一个程序总归是有一定的功能，人们用程序来完成任务。通常运行程序都需要一些输入，处理完后还会有一些输出。功能多且强大，意味着面临的情况多种多样，就需要很多输入。要实现灵活方便，就需要对输入进行分析。

假设有很多人会用这个程序，而且又会经常使用该程序。那么，通过分析对程序使用情况进行分析统计，对输入进行如下归类：
1. 程序必要，用户不得更改；
2. 不同运行环境会有所不同，确定后基本不变；
3. 每次运行都有所不同，变化特别频繁；
4. 变化频率介于二者之间；

对于第一种情况，程序不能让用户更改。这些输入数据通常以文件的形式，在程序指定位置存储。例如在程序的当前目录下。

对于第二种情况，提供可修改的配置文件。在程序第一次安装时进行配置保存即可。

对于第三种情况，在启动程序时，通过命令行传入即可。

对于第四种情况，在配置文件中进行配置，同时还可以通过命令行传入，后者会覆盖前者。

本节主要介绍如何用`argparse`处理程序的命令行输出，结合后续配置文件读写`configparser`可以是实现程序的各种输入。

## `sys.argv`

在使用 `argparse` 之前，先介绍一下 `sys.argv`。当使用 Python 在命令运行程序时，在命令行输入的程序名及其参数会存储在`sys.argv`中：

In [1]:
import sys

print(type(sys.argv))

<class 'list'>


可以查看一下Jupyter Notebook中运行的kernel启动情况：

In [2]:
sys.argv

['C:\\Anaconda3\\lib\\site-packages\\ipykernel_launcher.py',
 '-f',
 'C:\\Users\\whwang\\AppData\\Roaming\\jupyter\\runtime\\kernel-b929e32b-7d89-4c21-ad78-9471bf7cd1b4.json']

下面编写一个打印命令行参数的程序：

In [3]:
%%writefile demo01.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""sys.argv demo"""
import sys

print(sys.argv)
for index, arg in enumerate(sys.argv):
    print(index, arg)

Overwriting demo01.py


In [4]:
!python demo01.py arg1 arg2 arg3

['demo01.py', 'arg1', 'arg2', 'arg3']
0 demo01.py
1 arg1
2 arg2
3 arg3


In [5]:
!python demo01.py arg1 arg2 = 'x' arg3 = 4

['demo01.py', 'arg1', 'arg2', '=', "'x'", 'arg3', '=', '4']
0 demo01.py
1 arg1
2 arg2
3 =
4 'x'
5 arg3
6 =
7 4


如上所示，直接使用 `sys.argv` 来分析获得输入，麻烦得很，而且很容易出错。不过，幸好有`argparse`库。

## 自省

使用`dir()`列出 `argparse` 内容：

In [7]:
print(dir(argparse))

['Action', 'ArgumentDefaultsHelpFormatter', 'ArgumentError', 'ArgumentParser', 'ArgumentTypeError', 'FileType', 'HelpFormatter', 'MetavarTypeHelpFormatter', 'Namespace', 'ONE_OR_MORE', 'OPTIONAL', 'PARSER', 'REMAINDER', 'RawDescriptionHelpFormatter', 'RawTextHelpFormatter', 'SUPPRESS', 'ZERO_OR_MORE', '_', '_ActionsContainer', '_AppendAction', '_AppendConstAction', '_ArgumentGroup', '_AttributeHolder', '_CountAction', '_HelpAction', '_MutuallyExclusiveGroup', '_StoreAction', '_StoreConstAction', '_StoreFalseAction', '_StoreTrueAction', '_SubParsersAction', '_UNRECOGNIZED_ARGS_ATTR', '_VersionAction', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__version__', '_collections', '_copy', '_ensure_value', '_get_action_name', '_os', '_re', '_sys', '_textwrap', 'ngettext']


在`argparse`模块中，最主要的就是提供的参数解析器类 `ArgumentParser`。使用`argparse`解析命令行参数的主要步骤：
1. 创建`ArgumentParser()`对象
2. 调用`add_argument()`方法添加预期参数
3. 使用`parse_args()`解析用户输入命令行参数

下面创建一个`ArgumentParser()`对象，并使用自省方法查看使用。

In [9]:
parser = argparse.ArgumentParser()

`parser`最常用的方法是`add_argument()`, 其语法:
```
ArgumentParser.add_argument(name or flags...
    [, action][, nargs][, const][, default][, type]
    [, choices][, required][, help][, metavar][, dest])
```

不定位置参数说明如下：
- `name`，位置参数名: 
- `flags`，可选参数标识，短选项、长选项

主要关键字参数：
- `dest`: 在返回`ArgumentParser.parse_args`方法返回对象中，对应参数的属性名。
- `default`: 在命令行中没有指定参数时，参数的默认值。
- `action`: 在命令行中指定参数时，要执行的操作，即如何读取参数。
- `type`: 用来指定参数的数据类型，默认读取参数都是字符串。
- `help`: 参数说明
- `metavar`: 在使用帮助信息中参数的名称
- `choices`: 参数的可选值列表
- `nargs`: 从命令中读取参数值的数目。
- `const`: action或nargs所使用的一个常量值
- `required`: 参数是否必须，对可选参数来说默认值为False。

`action`关键字参数对应的值包括：
- `store`，缺省值,存储参数值
- `store_const`,存`const`指定的值
- `store_true`，存为`True`
- `store_false`，存为`False`
- `append`，参数值添加到列表
- `append_const`，添加`const`指定的值到列表中
- `count`，存储参数出现次数
- `version`，打印版本信息

`parser`的方法 `parse_args()` 会返回解析结果，是`argparse.Namespace`对象，可以使用`.`来访问对象的属性。语法是：
```
parse_args(args=None, namespace=None)
```
参数包括：
- `args`，缺省是`sys.argv`
- `namespace`,用于存放参数的对象。缺省是空的`Namespace`对象。

## 快速应用

### 无参数

下面给出一个没有参数的命令行解析示例。

In [ ]:
%%writefile demo02.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""argparse demo"""
import argparse

parser = argparse.ArgumentParser()
args = parser.parse_args()

print(args)

运行示例，查看结果

In [ ]:
!python demo02.py

不要着急，试一试帮助

In [ ]:
!python demo02.py -h

输入一些错误情况

In [ ]:
!python demo02.py wronginput -w

由上可知，`argparse`模块缺省提供了如下功能：
- `--help`或`-h`的帮助选项，以及帮助说明；
- 对于错误输入会给出提示

### 位置参数

前面讲过，程序的一些输入参数必须输入。对此种情况，可以使用位置参数（positional arguments）。

In [ ]:
%%writefile demo03.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""argparse demo"""
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('posarg1')
parser.add_argument('posarg2', help="位置参数2", type=float)
parser.add_argument('posarg3', help="位置参数3", type=int)
args = parser.parse_args()

print(args)

其中使用了关键字：
- `help`，会在说明帮助中显示
- `type`，指定参数值类型。如果不正确会报错误

查看程序的帮助信息

In [ ]:
!python demo03.py -h

运行程序，并查看结果

In [ ]:
!python demo03.py xyz 3.1415 128

试一试错误输入

In [ ]:
!python demo03.py 128 str xyz

### 可选参数

可选参数（optional arguments），也称为选项，有点像函数中的关键字参数，在Linux系统程序的选项通常用如下方式：
- 短参数，`-h`，通常用的频率稍高些
- 长参数，`--help`，通常用的频率稍低些

选项后面可以指定值，也可以不指定。

下面创建包括短参数和长参数的程序示例

In [ ]:
%%writefile demo04.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""argparse demo"""
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('-a', action="store_true", default=False)
parser.add_argument('-b', action="store", dest="b")
parser.add_argument('-c', action="store", dest="number", type=int)
parser.add_argument('--witharg', action="store", default='default d')
args = parser.parse_args()
print(args)

在调用`add_argument()`，使用了关键字参数：
- `action`
- `dest`
- `default`

在命令行中，指定每个选项

In [ ]:
!python demo04.py -a -bval -c 2 --witharg xyz

再比较一下，不指定选项的结果。

In [ ]:
!python demo04.py

可以看出，如果在命令行中不指定选项，则选项参数为`None`。可以使用`default`设置缺省值。

也可以像帮助选项那样，同时提供短参数和长参数。

In [ ]:
%%writefile demo05.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""argparse demo"""
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('-a', '--longa', action='store_true', default=False)
parser.add_argument('-b', '--longb', action='store', dest='longb')
parser.add_argument('-c', '--longc', action='store', dest='number', type=int)
args = parser.parse_args()
print(args)

运行程序，查看程序帮助信息

In [ ]:
!python demo05.py --help

指定选项或不指定选项，运行程序，比较运行结果。

In [ ]:
!python demo05.py

In [ ]:
!python demo05.py --longa -b 123 -c=123

### 参数混搭

一个命令行程序，通常有少量的定位参数和较多的可选参数。

In [ ]:
%%writefile demo06.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""argparse demo"""
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('-a', '--longa', action='store_true', default=False)
parser.add_argument('-b', '-longb', action='store', dest='longb')
parser.add_argument('-c', '--longc', action='store', dest='number', type=int)
parser.add_argument('inpfile')
args = parser.parse_args()
print(args)

运行程序，查看帮助信息

In [ ]:
!python demo06.py -h

运行程序，查看结果。

In [ ]:
!python demo06.py xxx.txt

## 更多说明

### `add_argument()`用法

#### `action`的使用

回顾一下，`action`关键字参数作用是对指定参数进行读取处理，并存到`dest`指定的属性中。`action`关键字参数对应的值有多个下面分别介绍。

缺省值为`store`，把参数指定的值存储到命名空间（Namespace）中。

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo')
argv = '--foo 1'.split()
parser.parse_args(argv)

当指定为`store_const`值时，会存储`const`参数传入值。

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='store_const', const=42)
argv = '--foo'.split()
parser.parse_args(argv)

当指定为`store_true`或`store_false`值时，会存储`True`或`False`值。

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='store_true')
parser.add_argument('--bar', action='store_false')
parser.add_argument('--baz', action='store_false')
parser.parse_args('--foo --bar'.split())

当指定为`append`值时，会把指定的参数添加到列表中，并存到命名空间中去。

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='append')
parser.parse_args('--foo 1 --foo 2'.split())

当指定为`append_const`值时，会把`const`传入值添加到列表，并存到`dest`指的属性中。

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--str', dest='types', action='append_const', const=str)
parser.add_argument('--int', dest='types', action='append_const', const=int)
parser.parse_args('--str --int'.split())

当指定为`count`值时，会对输入参数进行计数，并把结果存储。

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--verbose', '-v', action='count')
parser.parse_args(['-vvv'])

当指定为`version`值，会把`version`传入值打印出来，并退出程序。

In [ ]:
%%writefile demo07.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""argparse demo"""
import argparse


__version__ = '0.1.1'

parser = argparse.ArgumentParser()
parser.add_argument('-v', '--version', action='version', version=__version__)
args = parser.parse_args()

运行程序，检查帮助信息。

In [ ]:
!python demo07.py -h

运行程序，查看版本信息。

In [ ]:
!python demo07.py -v

#### `nargs`

可以使用关键参数`nargs`来指定选项值的数量，有如下可选值：  

| Value  |  说明|
|:-------|:------|
|N	| N个选项 |
|?	| 零个或1个选项值 |
|*	| 零个或任意多个选项值 |
|+	| 1个或任意多个选项 |

当指定为具体数字时，选项值数目必须匹配。

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', nargs=3)
parser.add_argument('bar', nargs=1)
argv = 'c --foo a b c'.split()
parser.parse_args(argv)

当指定为`?`时，表示要么指定一个选项值要么没有。

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', nargs='?', const='c', default='d')
parser.add_argument('bar', nargs='?', default='d')
argv = 'XX --foo a'.split()
parser.parse_args(argv)

当指定为`*`时，选项值数目可以是0，可以是1个，也可以任意多个。

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', nargs='*')
parser.add_argument('--bar', nargs='*')
parser.add_argument('baz', nargs='*')
argv = 'a b --foo --bar 1 2 x y'.split()
parser.parse_args(argv)

当指定为`+`时，表示选项要指定任意多个选项值，至少有一个。如果没有指定选项值会出错退出。

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('foo', nargs='+')
argv = 'a'.split()
print(parser.parse_args(argv))
argv = 'a b c'.split()
print(parser.parse_args(argv))
argv = ''.split()
parser.parse_args(argv)

#### `type`

`type`用来指定参数的数据类型，默认读取参数都是字符串。

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', type=int)
parser.add_argument('--bar', type=float)
parser.add_argument('--file', type=open)
argv = '--foo 1 --bar 2 --file demo01.py'.split()
parser.parse_args(argv)

#### `choices`

`choices`指定一个列表，参数值必须该列表中取值，否则会出错。

In [ ]:
parser = argparse.ArgumentParser()
msgnames = ['debug', 'info', 'warning', 'error', 'critical']
parser.add_argument('-m', '--msglev', choices=msgnames)
argv = '-m debug'.split()
print(parser.parse_args(argv))
argv = '-m x'.split()
print(parser.parse_args(argv))

### `ArgumentParser` 更多用法

要进行命令行解析，首先需要创建`ArgumentParser()`对象。在创建`ArgumentParser()`对象，可以使用关键字参数，来进行更多定制。

关键词参数包括：
- `prog`：程序名，缺省值是`sys.argv[0]`。
- `usage`：程序使用说明，缺省是自行创建。
- `description`： 在帮助前信息添加程序描述。
- `epilog`： 在帮助信息后显示内容
- `parents`： `ArgumentParser`对象列表。
- `formatter_class`： 帮助信息输出格式类。
- `prefix_chars`： 选项前缀，默认值是`-`。
- `fromfile_prefix_chars`： 前缀字符，放在文件名前，缺省是`None`。
- `argument_default`： 选项缺省值全局设置。
- `conflict_handler`： 冲突解决策略。
- `add_help`： 是否增加选项`-h/--help`，缺省是`True`
- `allow_abbrev`： Allows l

####  `description` 与`epilog`

使用`description` 与`epilog`在帮助信息中增加内容。

In [ ]:
%%writefile demo08.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""argparse demo"""
import argparse


__version__ = '0.1.1'

description = "程序用来演示version的用法"
epilog = "程序还不错吧"

parser = argparse.ArgumentParser(description=description, epilog=epilog)
parser.add_argument('-v', '--version', action='version', version=__version__)
args = parser.parse_args()

运行程序，比较与`demo07.py`的帮助信息差异

In [ ]:
!python demo08.py -h

#### `prefix_chars`

在Windows系统下，DOS命令的参数选项前缀不是`-`，而是使用`/`。故可以指定`prefix_chars='/'`来实现Dos风格的命令。

In [ ]:
%%writefile demo09.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""argparse demo"""
import argparse


__version__ = '0.1.1'

parser = argparse.ArgumentParser(prefix_chars='/')
parser.add_argument('/s', action='store_true', default=False)
parser.add_argument('/f', action='store')
args = parser.parse_args()
print(args)

运行程序，查看帮助信息。

In [ ]:
!python demo09.py /h

运行程序，查看参数解析结果。

In [ ]:
!python demo09.py

## 应用实例

传入多个整数（至少有一个），使用`--method`指定计算方法，缺省是求和。

In [ ]:
%%writefile calc.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""argparse demo"""
import argparse


__version__ = '1.0.0'

def main():
    """main entry"""
    description = "对多个整数进行累计运算"
    parser = argparse.ArgumentParser(description=description)
    parser.add_argument('integers', metavar='N', type=int, nargs='+',
                        help='an integer for the accumulator')
    parser.add_argument('--method', choices=['sum', 'min', 'max'], default='sum',
                        help='a math method for accumulator')
    parser.add_argument('-v', '--version', action='version', version=__version__)
    args = parser.parse_args()

    result = None
    if args.method == 'sum':
        result = sum(args.integers)
    elif args.method == 'max':
        result = max(args.integers)
    elif args.method == 'min':
        result = min(args.integers)

    print(result)
    
if __name__ == '__main__':
    main()

运行程序，检查帮助信息

In [ ]:
!python calc.py -h

运行程序，查看结果。

In [ ]:
!python calc.py 1 2 3

In [ ]:
!python calc.py --method  max 1 2 3